In [97]:
import pyodbc 
import pandas as pd

In [113]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=AAPL-L-627\SQLEXPRESS;'
                      'Database=test;'
                      'Trusted_Connection=yes;')
co = conn.cursor()

In [151]:
### reading the dataframe
df = pd.read_excel(r'D:\\Book1.xlsx')

In [152]:
df

,Col1,Col2,Col3,Col4,Col5
0,A,100,100,200,45
1,B,1,2,2,5
2,C,200,200,200,200
3,D,5,8,10,3
4,E,8,7,1,3
5,F,2,7,6,6
6,G,2,5,3,10
7,H,6,0,9,4
8,I,7,7,10,3
9,J,6,8,1,10


In [94]:
table = 'Table1'
key_id = 'Col1'

In [125]:
def append(df, c,table_name,key):
    


    columns_list = df.columns.tolist()
    columns_list_query = f'({(",".join(columns_list))})'
    sr_columns_list = [f'Source.{i}' for i in columns_list]
    sr_columns_list_query = f'({(",".join(sr_columns_list))})'
    up_columns_list = [f'{i}=Source.{i}' for i in columns_list]
    up_columns_list_query = f'{",".join(up_columns_list)}'

    rows_to_insert = [row.tolist() for idx, row in df.iterrows()]
    rows_to_insert = str(rows_to_insert).replace('[', '(').replace(']', ')')[1:][:-1]
    

    query = f"MERGE INTO {table_name} as Target \
            USING (SELECT * FROM \
           (VALUES {rows_to_insert}) \
           AS s {columns_list_query}\
           ) AS Source \
           ON Target.{key}=Source.{key}\
           WHEN NOT MATCHED by Target THEN \
           INSERT {columns_list_query} VALUES {sr_columns_list_query} \
           WHEN MATCHED THEN \
           UPDATE SET {up_columns_list_query};"
    c.execute(query)
    c.commit()

In [126]:
append(df,co,table,key_id)